<a href="https://colab.research.google.com/github/Shohreh2017/Coursera_Capstone/blob/master/Houston_Suburb_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A Guide to choose your Favorite Neighborhood in Houston (Texas)!**
### **This code is the assignment related to IBM DataScience capstone project.** 

In [2]:
# I am Using google colab instead of my own machine application
# importing the google.colab and mounting my google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import the required libraries

In [3]:
import pandas as pd
import numpy as np

# 1. Read the data files and store them in dataframe 

In [4]:
# Read the datasets and store them in a dataframe
data_path='/content/drive/My Drive/Colab Notebooks/data'

# read the texas zip code data
texas_zipCodes=pd.read_csv(data_path+'/texas_City_lat_long.csv')

# read Houston suburb city data
HoustonSuburb_city = pd.read_csv(data_path+'/selected_suburb_cities.csv')

# check out the dataframe 
texas_zipCodes.head()   

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,75087,Rockwall,TX,32.933820,-96.454460,-6,1,32.933820
1,78879,Rio Frio,TX,29.628266,-99.738470,-6,1,29.628266
2,79789,Wink,TX,31.753101,-103.157370,-6,1,31.753101
3,75403,Greenville,TX,33.218505,-96.048665,-6,1,33.218505
4,78108,Cibolo,TX,29.574127,-98.233080,-6,1,29.574127


In [5]:
# check the dimension of the data frame
print(texas_zipCodes.shape)

(2575, 8)


In [7]:
# check the data frame for Houston and some of its suburbs
HoustonSuburb_city[1:10]

,City
1,Pearland
2,Sugar Land
3,Spring
4,Missouri City
5,Atascocita
6,Bellaire
7,Humble
8,Katy
9,Tomball


## Clean up the dataframe And subset the texas cities to get only Houston and suburbs

In [8]:
# keep the important columns only
texas_zipCodes=texas_zipCodes[['Zip','City','Latitude','Longitude']]

# subset the texas cities to get only Houston and suburb zipcode, latitude and longitude
Houston_suburb_list=HoustonSuburb_city['City'].tolist()
Houston_suburb_df=texas_zipCodes[texas_zipCodes['City'].isin(Houston_suburb_list)]

# reset the index
Houston_suburb_df=Houston_suburb_df.reset_index(drop=True)

# check the dataframe dimension
Houston_suburb_df

,Zip,City,Latitude,Longitude
0,77373,Spring,30.056394,-95.389610
1,77005,Houston,29.717529,-95.428210
2,77735,Tomball,30.095391,-95.628023
3,77088,Houston,29.879213,-95.450280
4,77066,Houston,29.959439,-95.496940
...,...,...,...,...
245,77008,Houston,29.798777,-95.409510
246,77238,Houston,29.833990,-95.434241
247,77086,Houston,29.920981,-95.495560
248,77248,Houston,29.833990,-95.434241


### Check if any data is missing 

In [9]:
# Check if we have any NA values in the dataset
Houston_suburb_df.isna().sum()

Zip          0
City         0
Latitude     0
Longitude    0
dtype: int64

In [10]:
# find the frequency of each city in the dataset
Houston_suburb_df['City'].value_counts()

Houston          182
Spring            12
Pasadena           8
Humble             7
Katy               6
Victoria           5
Sugar Land         4
Tomball            3
Pearland           3
Cypress            3
Baytown            3
Stafford           2
Missouri City      2
Bellaire           2
Alief              1
North Houston      1
Hufsmith           1
Brazoria           1
Barker             1
Fresno             1
South Houston      1
Webster            1
Name: City, dtype: int64

In [11]:
#Remove the cities with less than 3 data records
Houston_suburb_df=Houston_suburb_df.groupby('City').filter(lambda x: len(x)>3)

Houston_suburb_df.shape

(224, 4)

In [12]:
# number of unique cities in the dataframe
len(Houston_suburb_df['City'].unique())

7

## Use geopy library to get the latitude and longitude values of Houston

In [13]:
from geopy.geocoders import Nominatim

address = 'Houston, TX'

geolocator = Nominatim(user_agent="trn_explorer")
location = geolocator.geocode(address)
Houston_latitude = location.latitude
Houston_longitude = location.longitude
print('The geograpical coordinate of Houston are {}, {}.'.format(Houston_latitude, Houston_longitude))

The geograpical coordinate of Houston are 29.7589382, -95.3676974.


## Create a map of Houston with Houston and suburbs superimposed on top

In [14]:
import folium

# create map of Houston using latitude and longitude values
map_Houston = folium.Map(location=[Houston_latitude, Houston_longitude], zoom_start=10)

# add markers to map
for lat, lng, Zip, City in zip(Houston_suburb_df['Latitude'], Houston_suburb_df['Longitude'], Houston_suburb_df['Zip'], Houston_suburb_df['City']):
    label = '{}, {}'.format(City, Zip)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Houston)  
    
map_Houston

In [15]:
# further refining the dataframe (Remove the data related to Victoria which is ver far away from Houston)
Houston_suburb_df=Houston_suburb_df[Houston_suburb_df['City']!='Victoria']

# check the dimension
Houston_suburb_df.shape

(219, 4)

## Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'ZL4LHHA554B0U152HSD14XXNH01VSCTPH2WGXO0AJG5JTFZ2' # your Foursquare ID
CLIENT_SECRET = 'EGSSU2XYALBISLTL30ZLCSYLBLTUMEALLSHUIWDIBPOYG0X3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZL4LHHA554B0U152HSD14XXNH01VSCTPH2WGXO0AJG5JTFZ2
CLIENT_SECRET:EGSSU2XYALBISLTL30ZLCSYLBLTUMEALLSHUIWDIBPOYG0X3


## Define a function that extracts the category of the venue

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# 2. Explore Houston Suburbs

### Create a function to extract the venues and category of the venue for all available data points in different cities

In [17]:
import requests

def getNearbyVenues(names, city, latitudes, longitudes, radius=1000,LIMIT=100):
    
    venues_list=[]
    for name, city, lat, lng in zip(names, city, latitudes, longitudes):
        # print(city)           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            city, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip_Code', 
                  'City',
                  'Zip_code_Latitude', 
                  'Zip_code_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

## For each Zip code within Houston and selected suburbs run the function and create a new dataframe called *houston_suburb_venues*

In [18]:
houston_suburb_venues = getNearbyVenues(names=Houston_suburb_df['Zip'],
                                        city=Houston_suburb_df['City'],
                                        latitudes=Houston_suburb_df['Latitude'],
                                        longitudes=Houston_suburb_df['Longitude']
                                        )

In [19]:
# check out the features of the df
Houston_suburb_df.columns

Index(['Zip', 'City', 'Latitude', 'Longitude'], dtype='object')

## Check the size of the resulting dataframe

In [20]:
print(houston_suburb_venues.shape)
houston_suburb_venues.head()

(7188, 8)


,Zip_Code,City,Zip_code_Latitude,Zip_code_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,77373,Spring,30.056394,-95.38961,La Presa Mexican Restaurant,30.051099,-95.388955,Mexican Restaurant
1,77373,Spring,30.056394,-95.38961,Chychos Pollos Al Carbon,30.048978,-95.387041,Food Truck
2,77373,Spring,30.056394,-95.38961,Tequila Patio,30.050139,-95.388583,Mexican Restaurant
3,77373,Spring,30.056394,-95.38961,White Flamingo Restaurant,30.058154,-95.398861,Breakfast Spot
4,77373,Spring,30.056394,-95.38961,SUBWAY,30.047921,-95.387498,Sandwich Place


### Check how many venues were returned for each city

In [21]:
houston_suburb_venues.groupby('City').count()


,Zip_Code,Zip_code_Latitude,Zip_code_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
City,,,,,,,
Houston,6292,6292,6292,6292,6292,6292,6292
Humble,252,252,252,252,252,252,252
Katy,164,164,164,164,164,164,164
Pasadena,211,211,211,211,211,211,211
Spring,247,247,247,247,247,247,247
Sugar Land,22,22,22,22,22,22,22


## Find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} unique categories.'.format(len(houston_suburb_venues['Venue_Category'].unique())))

There are 319 unique categories.


### Unify all types of Restaurants and all types of Bars as single categories, Resturant and Bar.


In [24]:
match_word_list=['Restaurant','Bar']
count_list=[]

for match_word in match_word_list:
  count_matchWord=0
  i=0
  for item in houston_suburb_venues['Venue_Category']:
    i=i+1     
    if match_word in item:
      # count the number of items which include Restaurant or Bar
      count_matchWord=count_matchWord+1
      houston_suburb_venues.loc[i-1,'Venue_Category']=match_word

  count_list.append(count_matchWord)

count_list


[1054, 419]

In [26]:
# check the number of unique categories of the returned venues
print('There are {} unique categories.'.format(len(houston_suburb_venues['Venue_Category'].unique())))

There are 254 unique categories.


# 3.Analyze the venues in each Suburb

In [27]:
# one hot encoding
houston_sub_onehot = pd.get_dummies(houston_suburb_venues[['Venue_Category']], prefix="", prefix_sep="")

# add Zip code, City, latitude and lonitude columns back to the dataframe    
houston_sub_onehot['Zip_Code'] = houston_suburb_venues['Zip_Code'] 
houston_sub_onehot['City'] = houston_suburb_venues['City']
houston_sub_onehot['Latitude'] = houston_suburb_venues['Zip_code_Latitude']
houston_sub_onehot['Longitude'] = houston_suburb_venues['Zip_code_Longitude']

# move Zip_Code and City column to the first and second column
fixed_columns = [houston_sub_onehot.columns[-4]] +[houston_sub_onehot.columns[-3]]+[houston_sub_onehot.columns[-2]] +[houston_sub_onehot.columns[-1]]+ list(houston_sub_onehot.columns[:-4])
houston_sub_onehot = houston_sub_onehot[fixed_columns]

houston_sub_onehot.head()

,Zip_Code,City,Latitude,Longitude,ATM,Accessories Store,Advertising Agency,Airport,Airport Lounge,Airport Service,Airport Terminal,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,...,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Swim School,Taco Place,Tanning Salon,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Theater,Thrift / Vintage Store,Toll Booth,Toy / Game Store,Trail,Travel & Transport,Travel Agency,Vehicle Inspection Station,Video Game Store,Video Store,Warehouse Store,Weight Loss Center,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,77373,Spring,30.056394,-95.38961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,77373,Spring,30.056394,-95.38961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,77373,Spring,30.056394,-95.38961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,77373,Spring,30.056394,-95.38961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,77373,Spring,30.056394,-95.38961,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
# examine the new dataframe size
houston_sub_onehot.shape

(7188, 258)

## Group rows by Zip_Code by taking the mean of the frequency of occurrence of each category

In [29]:
houston_suburb_grouped = houston_sub_onehot.groupby('Zip_Code').mean().reset_index()
houston_suburb_grouped.head()

,Zip_Code,Latitude,Longitude,ATM,Accessories Store,Advertising Agency,Airport,Airport Lounge,Airport Service,Airport Terminal,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,...,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Swim School,Taco Place,Tanning Salon,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Theater,Thrift / Vintage Store,Toll Booth,Toy / Game Store,Trail,Travel & Transport,Travel Agency,Vehicle Inspection Station,Video Game Store,Video Store,Warehouse Store,Weight Loss Center,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,77000,29.711257,-95.304936,0.037037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.037037,0.0,0.037037,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.0,0.0,0.0,0.0
1,77001,29.813142,-95.309789,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,77002,29.755578,-95.365310,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.010000,0.00,0.000000,0.0,0.120000,0.0,0.02,0.01,0.01,0.010000,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,...,0.01,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.01,0.0,0.0,0.000000,0.02,0.0,0.01,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,77003,29.749278,-95.347410,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.016129,0.0,0.0,0.0,0.016129,0.00,0.016129,0.0,0.145161,0.0,0.00,0.00,0.00,0.048387,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.048387,0.00,0.0,0.0,0.016129,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,77004,29.728779,-95.365700,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.05,0.050000,0.0,0.100000,0.0,0.00,0.00,0.00,0.000000,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [30]:
# convert the Zip_Code column into string
houston_suburb_grouped['Zip_Code']=houston_suburb_grouped['Zip_Code'].apply(str)

# confirm the new size
houston_suburb_grouped.shape

(217, 257)

### Find the matching city for each zip code (to be used in Cluster Analysis section)

In [31]:
# Find the corressponding city to the zip codes where the venues are found
location_df=houston_suburb_venues[['Zip_Code','City']]

# only keep the unique zip codes
location_df=location_df.drop_duplicates(['Zip_Code'])

location_df

,Zip_Code,City
0,77373,Spring
24,77005,Houston
44,77088,Houston
49,77066,Houston
57,77203,Houston
...,...,...
6959,77008,Houston
7059,77238,Houston
7102,77086,Houston
7135,77248,Houston


In [32]:
# convert the Zip_code comumn into string
location_df['Zip_Code']=location_df['Zip_Code'].apply(str)

### Create a Function to sort the venues in descending order

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Create a new dataframe which includes 7 most frequent venues for each zipcode

In [35]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip_Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most_Common_Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most_Common_Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Zip_Code'] = houston_suburb_grouped['Zip_Code']

for ind in np.arange(houston_suburb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(houston_suburb_grouped.iloc[ind, 3:], num_top_venues)

neighborhoods_venues_sorted.head()


,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
0,77000,Restaurant,Discount Store,Smoke Shop,Food Truck,Taco Place,Insurance Office,Lawyer
1,77001,Restaurant,Gas Station,Discount Store,Sandwich Place,Fried Chicken Joint,Department Store,Burger Joint
2,77002,Restaurant,Bar,Hotel,Gym,Park,Burger Joint,Pizza Place
3,77003,Restaurant,Bar,Pizza Place,Food Truck,Coffee Shop,Brewery,Soccer Stadium
4,77004,Restaurant,Food,Bar,Lounge,Theater,Food Service,Lawyer


# 4. Cluster Houston and Suburb zip codes based on the most frequent venues Using K-Means Clustering

In [36]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 6

houston_sub_grouped_clust_df = houston_suburb_grouped.drop(['Zip_Code','Latitude', 'Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(houston_sub_grouped_clust_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 3, 0, 1, 1, 3, 3, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 3, 0, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 3, 2, 2, 1, 0, 1, 4, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 3, 1, 0, 5, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 4, 0, 2, 0,
       1, 3, 1, 1, 1, 0, 3, 0, 0, 0, 3, 3, 0, 1, 0, 1, 0, 1, 1],
      dtype=int32)

### Create a new dataframe that includes the clusters as well as the top 7 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

# insert the location data to the dataframe
neighborhoods_venues_sorted.insert(1, 'Latitude', houston_suburb_grouped['Latitude'])
neighborhoods_venues_sorted.insert(2, 'Longitude', houston_suburb_grouped['Longitude'])

houston_sub_clust_merged = neighborhoods_venues_sorted.copy()

# change the column to integer
houston_sub_clust_merged['Cluster_Labels']=houston_sub_clust_merged['Cluster_Labels'].astype(int)

houston_sub_clust_merged.head() # check the last columns!

,Cluster_Labels,Latitude,Longitude,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
0,1,29.711257,-95.304936,77000,Restaurant,Discount Store,Smoke Shop,Food Truck,Taco Place,Insurance Office,Lawyer
1,1,29.813142,-95.309789,77001,Restaurant,Gas Station,Discount Store,Sandwich Place,Fried Chicken Joint,Department Store,Burger Joint
2,1,29.755578,-95.365310,77002,Restaurant,Bar,Hotel,Gym,Park,Burger Joint,Pizza Place
3,1,29.749278,-95.347410,77003,Restaurant,Bar,Pizza Place,Food Truck,Coffee Shop,Brewery,Soccer Stadium
4,1,29.728779,-95.365700,77004,Restaurant,Food,Bar,Lounge,Theater,Food Service,Lawyer


In [38]:
# add "City" to the above dataframe by merging with location_df
houston_sub_clust_merged=houston_sub_clust_merged.merge(location_df, on='Zip_Code')


# move Zip_Code and City column to the first and second column
fixed_columns2 = [houston_sub_clust_merged.columns[-1]]+ list(houston_sub_clust_merged.columns[:-1])
houston_sub_clust_merged = houston_sub_clust_merged[fixed_columns2]

houston_sub_clust_merged

,City,Cluster_Labels,Latitude,Longitude,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
0,Houston,1,29.711257,-95.304936,77000,Restaurant,Discount Store,Smoke Shop,Food Truck,Taco Place,Insurance Office,Lawyer
1,Houston,1,29.813142,-95.309789,77001,Restaurant,Gas Station,Discount Store,Sandwich Place,Fried Chicken Joint,Department Store,Burger Joint
2,Houston,1,29.755578,-95.365310,77002,Restaurant,Bar,Hotel,Gym,Park,Burger Joint,Pizza Place
3,Houston,1,29.749278,-95.347410,77003,Restaurant,Bar,Pizza Place,Food Truck,Coffee Shop,Brewery,Soccer Stadium
4,Houston,1,29.728779,-95.365700,77004,Restaurant,Food,Bar,Lounge,Theater,Food Service,Lawyer
...,...,...,...,...,...,...,...,...,...,...,...,...
212,Pasadena,0,29.695028,-95.157980,77503,Baseball Field,Convenience Store,Automotive Shop,Bakery,Soccer Field,Video Store,Restaurant
213,Pasadena,1,29.648780,-95.188130,77504,Restaurant,Discount Store,Video Store,Gas Station,Bar,Bank,Music Store
214,Pasadena,0,29.650492,-95.146320,77505,Cosmetics Shop,Restaurant,Coffee Shop,Mobile Phone Shop,Department Store,Clothing Store,Supplement Shop
215,Pasadena,1,29.705678,-95.202160,77506,Restaurant,Home Service,Shoe Store,Convenience Store,Video Store,Snack Place,BBQ Joint


In [40]:
# convert the 'Zip_Code' and 'Cluster_Labels' to integers
houston_sub_clust_merged['Zip_Code']=houston_sub_clust_merged['Zip_Code'].astype(int)
houston_sub_clust_merged['Cluster_Labels']=houston_sub_clust_merged['Cluster_Labels'].astype(int)

## Visualize the resulting clusters on the map

In [41]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[Houston_latitude, Houston_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, zip, cluster in zip(houston_sub_clust_merged['Latitude'], 
                                  houston_sub_clust_merged['Longitude'],
                                  houston_sub_clust_merged['Zip_Code'],
                                  houston_sub_clust_merged['Cluster_Labels']):
    label = folium.Popup(str(zip) + ' Cluster_' + str(cluster), parse_html=True)

    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

map_clusters

# 5. Examine Clusters
### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

### Cluster-1

In [42]:
houston_sub_clust_1=houston_sub_clust_merged.loc[houston_sub_clust_merged['Cluster_Labels'] == 0, houston_sub_clust_merged.columns[[0] + list(range(4, houston_sub_clust_merged.shape[1]))]]
houston_sub_clust_1

,City,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
5,Houston,77005,Park,Pharmacy,Baseball Field,Kids Store,Supplement Shop,Cafeteria,Taco Place
14,Houston,77014,Buffet,Hotel,BBQ Joint,Golf Course,Gym,Bakery,Convenience Store
16,Houston,77016,Park,Spa,Food,Clothing Store,Fried Chicken Joint,Other Repair Shop,Electronics Store
17,Houston,77017,Baseball Field,Discount Store,Grocery Store,Outdoors & Recreation,Video Store,Casino,Doctor's Office
18,Houston,77018,Restaurant,Bar,Pizza Place,Bank,Gas Station,Liquor Store,Cosmetics Shop
...,...,...,...,...,...,...,...,...,...
206,Katy,77492,Spa,Bar,Restaurant,Pizza Place,Gas Station,Discount Store,Cosmetics Shop
207,Katy,77493,Playground,Park,Construction & Landscaping,Mini Golf,Home Service,Miscellaneous Shop,Gym
210,Pasadena,77501,Spa,Bar,Restaurant,Pizza Place,Gas Station,Discount Store,Cosmetics Shop
212,Pasadena,77503,Baseball Field,Convenience Store,Automotive Shop,Bakery,Soccer Field,Video Store,Restaurant


In [43]:
houston_sub_clust_1['City'].value_counts()

Houston       108
Spring          6
Humble          4
Katy            3
Pasadena        3
Sugar Land      1
Name: City, dtype: int64

In [44]:
# evaluate the frequency of the venues for the first 3 column
print(houston_sub_clust_1['1st Most_Common_Venue'].value_counts())
print(houston_sub_clust_1['2nd Most_Common_Venue'].value_counts())
print(houston_sub_clust_1['3rd Most_Common_Venue'].value_counts())

Spa                  88
Park                  5
Restaurant            5
Pizza Place           4
Bar                   3
Grocery Store         2
Baseball Field        2
Playground            2
Tennis Court          1
Scenic Lookout        1
Clothing Store        1
Burger Joint          1
Snack Place           1
Flea Market           1
Cosmetics Shop        1
River                 1
Liquor Store          1
Convenience Store     1
Buffet                1
Pharmacy              1
Flower Shop           1
Trail                 1
Name: 1st Most_Common_Venue, dtype: int64
Bar                           90
Restaurant                     5
Park                           3
Cosmetics Shop                 3
Pizza Place                    2
Discount Store                 2
Convenience Store              2
Home Service                   2
Tennis Court                   1
Pharmacy                       1
Music Venue                    1
Smoke Shop                     1
Medical Supply Store           1
G

### Cluster-2

In [45]:
houston_sub_clust_2=houston_sub_clust_merged.loc[houston_sub_clust_merged['Cluster_Labels'] == 1, houston_sub_clust_merged.columns[[0] + list(range(4, houston_sub_clust_merged.shape[1]))]]
houston_sub_clust_2

,City,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
0,Houston,77000,Restaurant,Discount Store,Smoke Shop,Food Truck,Taco Place,Insurance Office,Lawyer
1,Houston,77001,Restaurant,Gas Station,Discount Store,Sandwich Place,Fried Chicken Joint,Department Store,Burger Joint
2,Houston,77002,Restaurant,Bar,Hotel,Gym,Park,Burger Joint,Pizza Place
3,Houston,77003,Restaurant,Bar,Pizza Place,Food Truck,Coffee Shop,Brewery,Soccer Stadium
4,Houston,77004,Restaurant,Food,Bar,Lounge,Theater,Food Service,Lawyer
...,...,...,...,...,...,...,...,...,...
202,Sugar Land,77478,Restaurant,Spa,Bank,Bar,Baseball Field,Sandwich Place,Discount Store
211,Pasadena,77502,Grocery Store,Restaurant,Farmers Market,Fried Chicken Joint,Theater,Ice Cream Shop,Taco Place
213,Pasadena,77504,Restaurant,Discount Store,Video Store,Gas Station,Bar,Bank,Music Store
215,Pasadena,77506,Restaurant,Home Service,Shoe Store,Convenience Store,Video Store,Snack Place,BBQ Joint


In [46]:
houston_sub_clust_2['City'].value_counts()

Houston       62
Pasadena       4
Spring         4
Humble         2
Katy           2
Sugar Land     1
Name: City, dtype: int64

In [47]:
# evaluate the frequency of the venues for the first 3 column
print(houston_sub_clust_2['1st Most_Common_Venue'].value_counts())
print(houston_sub_clust_2['2nd Most_Common_Venue'].value_counts())
print(houston_sub_clust_2['3rd Most_Common_Venue'].value_counts())

Restaurant             69
Grocery Store           2
Art Gallery             1
Food                    1
Rental Car Location     1
Pizza Place             1
Name: 1st Most_Common_Venue, dtype: int64
Pizza Place                11
Bar                         9
Hotel                       6
Discount Store              5
Grocery Store               4
Liquor Store                3
Furniture / Home Store      2
Department Store            2
Coffee Shop                 2
Shop & Service              2
Food                        2
Restaurant                  2
Mobile Phone Shop           2
Home Service                2
Donut Shop                  1
Fried Chicken Joint         1
Intersection                1
Science Museum              1
Health & Beauty Service     1
Gym / Fitness Center        1
Ice Cream Shop              1
Food Truck                  1
Nightclub                   1
Movie Theater               1
Athletics & Sports          1
Pharmacy                    1
Gas Station           

### Cluster-3

In [48]:
houston_sub_clust_3=houston_sub_clust_merged.loc[houston_sub_clust_merged['Cluster_Labels'] == 2, houston_sub_clust_merged.columns[[0] + list(range(4, houston_sub_clust_merged.shape[1]))]]
houston_sub_clust_3

,City,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
68,Houston,77068,Golf Course,Lake,Arts & Crafts Store,Advertising Agency,Pool,Discount Store,Doctor's Office
69,Houston,77069,Golf Course,Home Service,Yoga Studio,Escape Room,Doctor's Office,Dog Run,Donut Shop
196,Spring,77389,Pool,Golf Course,Yoga Studio,Electronics Store,Discount Store,Doctor's Office,Dog Run


### Cluster-4

In [49]:
houston_sub_clust_4=houston_sub_clust_merged.loc[houston_sub_clust_merged['Cluster_Labels'] == 3, houston_sub_clust_merged.columns[[0] + list(range(4, houston_sub_clust_merged.shape[1]))]]
houston_sub_clust_4

,City,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
24,Houston,77024,Construction & Landscaping,Coffee Shop,Ice Cream Shop,Clothing Store,Business Service,Eye Doctor,Fabric Shop
28,Houston,77028,Fried Chicken Joint,Construction & Landscaping,Convenience Store,Food,Burger Joint,Restaurant,Yoga Studio
29,Houston,77029,Platform,Construction & Landscaping,Moving Target,Disc Golf,Farmers Market,Fabric Shop,Eye Doctor
47,Houston,77047,Construction & Landscaping,Dog Run,Restaurant,Home Service,Fried Chicken Joint,Hotel,Park
67,Houston,77067,Construction & Landscaping,Food,Gas Station,Home Service,Convenience Store,Restaurant,Business Service
91,Houston,77091,Moving Target,Construction & Landscaping,Rest Area,Burger Joint,Rental Service,Electronics Store,Dog Run
199,Humble,77396,Fried Chicken Joint,Convenience Store,Breakfast Spot,Gym,Moving Target,Yoga Studio,Donut Shop
204,Sugar Land,77487,Park,Construction & Landscaping,Burrito Place,Yoga Studio,Dog Run,Donut Shop,Dry Cleaner
208,Katy,77494,Construction & Landscaping,Athletics & Sports,Public Art,Yoga Studio,Escape Room,Dog Run,Donut Shop
209,Sugar Land,77496,Park,Construction & Landscaping,Burrito Place,Yoga Studio,Dog Run,Donut Shop,Dry Cleaner


In [50]:
houston_sub_clust_4['City'].value_counts()

Houston       6
Sugar Land    2
Humble        1
Katy          1
Name: City, dtype: int64

In [51]:
# evaluate the frequency of the venues for the first 3 columns
print(houston_sub_clust_4['1st Most_Common_Venue'].value_counts())
print(houston_sub_clust_4['2nd Most_Common_Venue'].value_counts())
print(houston_sub_clust_4['3rd Most_Common_Venue'].value_counts())

Construction & Landscaping    4
Fried Chicken Joint           2
Park                          2
Platform                      1
Moving Target                 1
Name: 1st Most_Common_Venue, dtype: int64
Construction & Landscaping    5
Dog Run                       1
Food                          1
Athletics & Sports            1
Convenience Store             1
Coffee Shop                   1
Name: 2nd Most_Common_Venue, dtype: int64
Burrito Place        2
Moving Target        1
Restaurant           1
Convenience Store    1
Gas Station          1
Rest Area            1
Ice Cream Shop       1
Public Art           1
Breakfast Spot       1
Name: 3rd Most_Common_Venue, dtype: int64


### Cluster-5

In [52]:
houston_sub_clust_5=houston_sub_clust_merged.loc[houston_sub_clust_merged['Cluster_Labels'] == 4, houston_sub_clust_merged.columns[[0] + list(range(4, houston_sub_clust_merged.shape[1]))]]
houston_sub_clust_5

,City,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
50,Houston,77050,Home Service,Yoga Studio,Discount Store,Financial or Legal Service,Field,Farmers Market,Fabric Shop
73,Houston,77073,Home Service,Park,Yoga Studio,Escape Room,Doctor's Office,Dog Run,Donut Shop
194,Spring,77387,Boat or Ferry,Home Service,Event Service,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner


### Cluster-6

In [53]:
houston_sub_clust_6=houston_sub_clust_merged.loc[houston_sub_clust_merged['Cluster_Labels'] == 5, houston_sub_clust_merged.columns[[0] + list(range(4, houston_sub_clust_merged.shape[1]))]]
houston_sub_clust_6

,City,Zip_Code,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue
94,Houston,77094,Park,Yoga Studio,Disc Golf,Field,Farmers Market,Fabric Shop,Eye Doctor
